In [6]:
from pathlib import Path
from omnibelt import load_json, save_json, save_txt, load_txt
from collections import Counter

In [2]:
root = Path('../data/backup/')
path = root / 'cladder-v1-q-balanced.json'
path.exists()

True

In [3]:
full = load_json(path)
models = load_json(root / 'cladder-v1-meta-models.json')
len(full), len(models)

(10112, 7064)

In [18]:
key = 'rung'
vocabs = {}
for entry in full:
	vocabs.setdefault(entry['meta'][key], []).extend(entry['question'].split())
	vocabs.setdefault(entry['meta'][key], []).extend(entry['given_info'].split())
	model = models[entry['meta']['model_id']]
	vocabs.setdefault(entry['meta'][key], []).extend(model['background'].split())
def _cleanup(w):
	w = w.lower()
	w = w.replace('?', '').replace('.', '').replace(',', '').replace('\'', '').replace('\"', '').replace('%', '').replace(':', '')
	# check if it's a number
	try:
		float(w)
		return 'NUM'
	except:
		pass
	return w
vocabs = {k: [_cleanup(w) for w in v] for k, v in vocabs.items()}
vocabs = {k: Counter(v) for k, v in vocabs.items()}
vocabs.keys()

dict_keys([3, 2, 1])

In [19]:
total = Counter()
for v in vocabs.values():
	total.update(v)
len(total)

539

In [7]:
full[0].keys()

dict_keys(['question_id', 'desc_id', 'given_info', 'question', 'answer', 'meta', 'reasoning'])

In [8]:
full[0]['meta'].keys()

dict_keys(['story_id', 'graph_id', 'mediators', 'polarity', 'groundtruth', 'query_type', 'rung', 'formal_form', 'given_info', 'estimand', 'treatment', 'outcome', 'model_id'])

In [7]:
simple = []

for entry in full:
	model = models[entry['meta']['model_id']]
	# if not model.get('nonsense', False):
	simple.append({
		'id': entry['question_id'],
		'prompt': f'{model["background"]} {entry["given_info"]} {entry["question"]}',
		'ground_truth': entry['answer'],
	})
len(simple)

10112

In [8]:
# save_txt('\n'.join([item['prompt'] for item in simple]), root / 'cladder-v1-prompts.txt')

5817039

In [25]:
models[:3]

[{'model_id': 0,
  'story_id': 'alarm',
  'graph_id': 'mediation',
  'spec_id': 0,
  'spec': {'X': [0.6, 0.8], 'V2': {'X': -0.2}, 'Y': {'X': 0.3, 'V2': 0.4}},
  'seed': 101,
  'builder': 'difficulty',
  'difficulty': 'easy',
  'equation_type': 'bernoulli',
  'background': 'Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Husband has a direct effect on wife and alarm clock. Wife has a direct effect on alarm clock.',
  'variable_mapping': {'Xname': 'husband',
   'X1': 'alarm set by husband',
   'X0': 'alarm not set by husband',
   'V2name': 'wife',
   'V21': 'alarm set by wife',
   'V20': 'alarm not set by wife',
   'Yname': 'alarm clock',
   'Y1': 'ringing alarm',
   'Y0': 'silent alarm'},
  'structure': 'X->V2,X->Y,V2->Y',
  'params': {'p(X)': 0.7887065011221108,
   'p(V2 | X)': [0.7416866188819118, 0.2351932407152126],
   'p(Y | X, V2)': [[0.08430222457648492, 0.5394610521458689],
    [0.40615

In [20]:
print(simple[0]['id'])

4


In [14]:
save_json(simple, root / 'cladder-v1-prompts.json')

In [4]:
from tqdm.notebook import tqdm
from efficiency.function import avg

In [5]:
import grammar_check
tool = grammar_check.LanguageTool('en-GB')
text = 'This are bad.'
matches = tool.check(text)
len(matches)

ModuleNotFoundError: No module named 'grammar_check'

In [ ]:
texts = [item['prompt'] for item in simple]
# grammatical errors
bar = tqdm(texts)
cnts = []
for text in bar:
cnt = count_grammar_mistakes(text)
cnts.append(cnt)
bar.set_description(str(avg(cnts)))

In [4]:
import language_tool_python

from tqdm.notebook import tqdm
from efficiency.function import avg

In [7]:
def count_grammar_mistakes(text):
   tool = language_tool_python.LanguageTool('en-US')
   matches = tool.check(text)
   return len(matches)

texts = [q['question'] for q in full[:10]] # TODO: Felix, can you exclude the nonsensical, which triggers grammatical errors? Then we can get the count of grammatical errors.
texts = ['These is cool.', 'This is cool.', 'This is cool to I and he.']
bar = tqdm(texts)
cnts = []
for text in bar:
   cnt = count_grammar_mistakes(text)
   cnts.append(cnt)
   bar.set_description(str(avg(cnts)))


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
cnts

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]